# sentiment_analysis

Crearé una función que tome como entrada una cadena de texto que represente a una empresa desarrolladora y devuelva la cantidad de reseñas positivas, negativas y neutrales. Para llevar a cabo esta tarea, se creará un archivo CSV con la información necesaria para realizar la consulta. Debido a que la función se desplegará en Render y cargar datasets pesados puede resultar poco práctico.

def sentiment_analysis( empresa desarrolladora : str ): Devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}



### Importación de Bibliotecas

In [2]:
import pandas as pd

### Lectura de Archivos

In [3]:
steam_games = pd.read_csv("..\\Datsets\\CSV\\steam_games.csv")
user_reviews = pd.read_csv("..\\Datsets\\CSV\\user_reviews.csv")

### Creación del CSV

Para realizar el análisis de sentimientos, se requiere combinar información de dos conjuntos de datos: steam_games y user_reviews. Se utilizarán las columnas developer de steam_games y sentiment_analysis de user_reviews. Además, se emplearán las columnas id de steam_games y item_id de user_reviews para concatenar los DataFrames.

In [4]:
#Concatenamos los dataframes
analisis_sentimientos_df = steam_games[['id', 'developer']].merge(user_reviews[['item_id', 'sentiment_analysis']], left_on='id', right_on='item_id', how='inner')
#Quitamos las columnas id e item_id porque no las vamos a ocupar
analisis_sentimientos_df = analisis_sentimientos_df.drop(columns=['id', 'item_id'])

In [5]:
analisis_sentimientos_df.head()

,developer,sentiment_analysis
0,Stainless Games Ltd,1
1,Valve,2
2,Valve,0
3,Valve,2
4,Valve,2


Vamos a ajustar los valores en la columna `sentiment_analysis` de la siguiente manera:

- 0 para representar "Negativo".
- 1 para representar "Neutral".
- 2 para representar "Positivo".

In [6]:
# Convertir los valores en la columna 'sentiment_analysis' a int
analisis_sentimientos_df['sentiment_analysis'] = analisis_sentimientos_df['sentiment_analysis'].astype(int)
#Cambiamos los valores por positivo , negativo y neutral
analisis_sentimientos_df['sentiment_analysis'] = analisis_sentimientos_df['sentiment_analysis'].replace({0: 'Negativo', 1: 'Neutral', 2: 'Positivo'})

In [7]:
analisis_sentimientos_df.head()

,developer,sentiment_analysis
0,Stainless Games Ltd,Neutral
1,Valve,Positivo
2,Valve,Negativo
3,Valve,Positivo
4,Valve,Positivo


Agrupar los datos por el campo developer. Contando las veces que aparece cada valor en la columna sentiment_analysis para cada desarrollador.

In [8]:
# Agrupar los datos y contar los valores
analisis_sentimientos_df = analisis_sentimientos_df.groupby('developer')['sentiment_analysis'].value_counts()


In [9]:
analisis_sentimientos_df.head()

developer                              sentiment_analysis
07th Expansion                         Neutral               2
                                       Negativo              1
10th Art Studio,Adventure Productions  Negativo              1
                                       Positivo              1
10tons Ltd                             Positivo              1
Name: count, dtype: int64

Los resultados estén en una estructura de índice múltiple. Para facilitar el análisis y la visualización, desapilaremos los índices.

In [10]:
# Desapilar los índices
analisis_sentimientos_df = analisis_sentimientos_df.unstack()
# Resetear el índice
analisis_sentimientos_df = analisis_sentimientos_df.reset_index()

In [11]:
analisis_sentimientos_df.head()

sentiment_analysis,developer,Negativo,Neutral,Positivo
0,07th Expansion,1.0,2.0,NaN
1,"10th Art Studio,Adventure Productions",1.0,NaN,1.0
2,10tons Ltd,NaN,NaN,1.0
3,11 bit studios,28.0,9.0,25.0
4,14° East,1.0,NaN,1.0


### Convertimos a CSV

In [12]:
#Convertimos a CSV
analisis_sentimientos_df.to_csv('..\\Datsets\\Archivos_API\\sentiment_analysis.csv', index=False)


In [13]:
def sentiment_analysis(developer):
    """
    Retorna un diccionario que muestra la cantidad de reseñas positivas, negativas y neutrales para un desarrollador específico.

    Parameters:
    - developer (str): El nombre del desarrollador para el cual se desea obtener el análisis de sentimientos.

    Returns:
    - dict: Un diccionario con el nombre del desarrollador como clave y una lista con la cantidad total de registros
            de reseñas de usuarios categorizados con un análisis de sentimiento como valor.
            Ejemplo de retorno: {'Valve': {'Negative': 182, 'Neutral': 120, 'Positive': 278}}

            En caso de que no se encuentren datos para el desarrollador especificado, se retorna un mensaje informativo.
            Ejemplo: "No se encontraron datos para el desarrollador 'NombreDelDesarrollador'".
    """
    developer_column = analisis_sentimientos_df["developer"].tolist()

    if developer in developer_column:
        index = developer_column.index(developer)
        sentiment_counts = analisis_sentimientos_df.iloc[index, 1:].to_dict()
        return {developer: sentiment_counts}
    else:
        return f"No se encontraron datos para el desarrollador '{developer}'"
